In [1]:
!pip install -q torchaudio omegaconf onnx onnxruntime


You should consider upgrading via the 'c:\users\c_v\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
from pydub import AudioSegment
import onnx
import torch
import onnxruntime
from omegaconf import OmegaConf
from moviepy.editor import *

In [2]:
from pydub.utils import make_chunks 


In [3]:
mp4_file = r'eng.mp4'     #mp4 location
mp3_file = r'neweng.mp3' #mp3 location

In [4]:
videoclip = VideoFileClip(mp4_file)

audioclip = videoclip.audio
audioclip.write_audiofile(mp3_file)

audioclip.close()
videoclip.close()


chunk:   0%|                                                                | 3/8316 [00:00<06:44, 20.54it/s, now=None]

MoviePy - Writing audio in neweng.mp3


MoviePy - Done.


In [5]:
sound = AudioSegment.from_mp3("C:/Users/C_v/Desktop/neweng.mp3")
sound.export("C:/Users/C_v/Desktop/neweng.wav", format="wav")


<_io.BufferedRandom name='C:/Users/C_v/Desktop/neweng.wav'>

In [6]:
myaudio = AudioSegment.from_file("neweng.wav", "wav") 
chunk_length_ms = 80000 # pydub calculates in millisec 
chunks = make_chunks(myaudio,chunk_length_ms) #Make chunks of one sec 
count=0
for i, chunk in enumerate(chunks): 
    count=count+1
    chunk_name = "{0}.wav".format(i) 
    print ("exporting", chunk_name) 
    chunk.export(chunk_name, format="wav") 
count

exporting 0.wav
exporting 1.wav
exporting 2.wav
exporting 3.wav
exporting 4.wav


5

In [9]:
# sound = AudioSegment.from_mp3("/Users/pphilemo/Documents/UMN/KDI/Spring/NewKDISeptember.mp3") #mp3 location
# sound.export("/Users/pphilemo/Documents/UMN/KDI/Spring/NewKDICapture.wav", format="wav") #wavlocation

In [ ]:

language = 'en' 


_, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models', model='silero_stt', language=language)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml', 'models.yml')
models = OmegaConf.load('models.yml')
available_languages = list(models.stt_models.keys())
assert language in available_languages

torch.hub.download_url_to_file(models.stt_models.en.latest.onnx, 'model.onnx', progress=True)
onnx_model = onnx.load('model.onnx')
onnx.checker.check_model(onnx_model)
ort_session = onnxruntime.InferenceSession('model.onnx')


Using cache found in C:\Users\C_v/.cache\torch\hub\snakers4_silero-models_master


In [ ]:
data=[]
for i in range(count):
    i=str(i)
    test_files = [i+'.wav'] #wavname
    print(test_files)
    batches = split_into_batches(test_files, batch_size=10)
    input = prepare_model_input(read_batch(batches[0]))
    
    onnx_input = input.detach().cpu().numpy()
    ort_inputs = {'input': onnx_input}
    ort_outs = ort_session.run(None, ort_inputs)
    decoded = decoder(torch.Tensor(ort_outs[0])[0])
    data.append(decoded)


In [ ]:
import pandas as pd


In [ ]:
df=pd.DataFrame(data,columns=['Text'])

In [ ]:
df.to_csv('text_data.csv')
# df.to_csv('/Users/pphilemo/Documents/UMN/KDI/Spring/text_data.csv')